<a href="https://colab.research.google.com/github/virf96/Chat-Bot/blob/master/DatosFaltantes_SustitucionModa_Sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sustitución por la moda | categoría más frecuente con Scikit-learn ==> SimpleImputer 

En la librería Scikit-learn hay una clase para manejar una gran variedad de métodos de sustitución.

El **SimpleImputer** es una clase que provee funcionalidad básica para la sustitución de valores ausentes, incluyendo:

- Sustitución por la media y la mediana para variables numéricas
- Sustitución por la categoría más frecuente para variables categóricas.
- Sustitución por valores arbitrarios para variables numéricas y categóricas.

### Ventajas

- Fácil de usar si se aplica a todo el dataframe
- Código mantenido por desarrolladores de Scikit-learn: buena calidad
- Rápida computación (usa NumPy para los cálculos)
- Permite usar grid-search (búsqueda en cuadrículas) para varios métodos de sustitución
- Permite usar diferentes valores para codificar ausencia de datos (se puede indicar si por ejemplo los valores nulos son np.nan, ceros, cadenas de caracteres vacías, u otros)

### Limitaciones

- Retorna un arreglo de NumPy en lugar de un dataframe de pandas, lo cual es inconveniente para el análisis de datos
- Necesita usar clases adicionales para seleccionar cuales variables se deben sustituir ==>
    - requiere líneas de código adicional
    - requiere ser usado con otras clases
    - no es tan sencillo de usar
    
### Más detalles acerca de los transformadores (transformers en inglés)

- [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer)
- [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)
- [Stackoverflow](https://stackoverflow.com/questions/54160370/how-to-use-sklearn-column-transformer)

## En este demo:

Vamos a aprender **sustitución por moda | categoría más frecuente usando Scikit-learn** usando los datos Ames House Price.

- Para bajar los datos, por favor referirse a la clase **Datasets** en la  **Sección 1** del curso.

### Nota: 
* 'Imputer' se deriva del verbo en inglés 'to impute' que quiere decir sustituir o reemplazar. Imputer es el objeto que completa la sustitución, de ahí el nombre dado a la clase.
* 'slicing' significa seleccionar conjuntos de datos (columnas/filas) de un ‘DataFrame’.



In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# estas son las clases para sustitutición con sklearn
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# dividir dataset
from sklearn.model_selection import train_test_split

In [2]:
# solo usaremos las siguientes variables en el demo:
# una mezcla de  variables categóricas y numéricas

cols_to_use = ['BsmtQual', 'FireplaceQu', 'MSZoning',
               'BsmtUnfSF', 'LotFrontage', 'MasVnrArea',
               'Street', 'Alley', 'SalePrice']

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# carguemos los datos

data = pd.read_csv('/content/drive/My Drive/datasets/houseprice.csv', usecols=cols_to_use)
print(data.shape)
data.head()

(1460, 9)


,MSZoning,LotFrontage,Street,Alley,MasVnrArea,BsmtQual,BsmtUnfSF,FireplaceQu,SalePrice
0,RL,65.0,Pave,NaN,196.0,Gd,150,NaN,208500
1,RL,80.0,Pave,NaN,0.0,Gd,284,TA,181500
2,RL,68.0,Pave,NaN,162.0,Gd,434,TA,223500
3,RL,60.0,Pave,NaN,0.0,TA,540,Gd,140000
4,RL,84.0,Pave,NaN,350.0,Gd,490,TA,250000


In [5]:
# revisemos los valores nulos

data.isnull().mean().sort_values(ascending=False)

Alley          0.937671
FireplaceQu    0.472603
LotFrontage    0.177397
BsmtQual       0.025342
MasVnrArea     0.005479
SalePrice      0.000000
BsmtUnfSF      0.000000
Street         0.000000
MSZoning       0.000000
dtype: float64

In [6]:
# separar datos en segmentos entrenamiento y prueba

# primero, separemos el target (SalePrice) del resto de las variables (features)
cols_to_use.remove('SalePrice')

X_train, X_test, y_train, y_test = train_test_split(data[cols_to_use], # solo las variables
                                                    data['SalePrice'], # el target
                                                    test_size=0.3, # el porcentaje de obs en el segmento de prueba
                                                    random_state=0) # para reproducir
X_train.shape, X_test.shape

((1022, 8), (438, 8))

In [9]:
# evaluemos el porcentaje de datos ausentes nuevamente
#El porcentaje de missing values a cambiado ligeramente ya que solo trabajamos con el 70% del set original
#El target no esta considerado
X_train.isnull().mean().sort_values(ascending=False)

Alley          0.939335
FireplaceQu    0.467710
LotFrontage    0.184932
BsmtQual       0.023483
MasVnrArea     0.004892
Street         0.000000
BsmtUnfSF      0.000000
MSZoning       0.000000
dtype: float64

### SimpleImputer en el conjunto de datos

In [10]:
# Ahora sustituyamos los valores faltantes con  SimpleImputer

# creemos una instancia de la clase SimpleImputer
# indicaremos que queremos sustituir con la
# categoría más frecuente

imputer = SimpleImputer(strategy='most_frequent')

# ajustamos el imputer al segmento de entrenamiento
# el imputer aprende cual es la moda para todas las variables, categóricas o no

imputer.fit(X_train[cols_to_use])

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='most_frequent', verbose=0)

In [11]:
# podemos ver los valores más frecuentes aprendidos asi:
imputer.statistics_

array(['TA', 'Gd', 'RL', 0, 60.0, 0.0, 'Pave', 'Pave'], dtype=object)

**Nota** el transformador aprende cual es el valor mas frecuente para las variables categóricas y numéricas

In [12]:
# podemos investigar los valores más frecuentes para corroborar que el imputer
# hizo un buen trabajo

X_train[cols_to_use].mode()

,BsmtQual,FireplaceQu,MSZoning,BsmtUnfSF,LotFrontage,MasVnrArea,Street,Alley
0,TA,Gd,RL,0,60.0,0.0,Pave,Pave


In [13]:
# ahora sustituyamos los datos ausentes en los segmentos de entrenamiento y prueba

# NOTA: los datos son devueltos como un numpy array!!

X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

X_train

array([['Gd', 'Gd', 'RL', ..., 573.0, 'Pave', 'Pave'],
       ['Gd', 'Gd', 'RL', ..., 0.0, 'Pave', 'Pave'],
       ['TA', 'Gd', 'RL', ..., 0.0, 'Pave', 'Pave'],
       ...,
       ['TA', 'Gd', 'RM', ..., 0.0, 'Pave', 'Pave'],
       ['Gd', 'TA', 'RL', ..., 18.0, 'Pave', 'Pave'],
       ['Gd', 'Gd', 'RL', ..., 30.0, 'Pave', 'Pave']], dtype=object)

In [14]:
# transformar el segmento de entrenamiento de vuelta en un dataframe:

pd.DataFrame(X_train, columns=cols_to_use).head()

,BsmtQual,FireplaceQu,MSZoning,BsmtUnfSF,LotFrontage,MasVnrArea,Street,Alley
0,Gd,Gd,RL,318,60,573,Pave,Pave
1,Gd,Gd,RL,288,60,0,Pave,Pave
2,TA,Gd,RL,162,50,0,Pave,Pave
3,TA,Gd,RL,356,60,0,Pave,Pave
4,TA,Gd,RL,0,60,0,Pave,Pave


### SimpleImputer: diferentes métodos en diferentes variables

In [15]:
# Separemos en segmentos para entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(data[cols_to_use],
                                                    data['SalePrice'],
                                                    test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((1022, 8), (438, 8))

In [16]:
# exploremps los valores nulos
X_train.isnull().mean().sort_values(ascending=False)

Alley          0.939335
FireplaceQu    0.467710
LotFrontage    0.184932
BsmtQual       0.023483
MasVnrArea     0.004892
Street         0.000000
BsmtUnfSF      0.000000
MSZoning       0.000000
dtype: float64

Para este demo, vamos a sustituir:

- variables categóricas con la categoría más frecuente
- variables numéricas con la media


In [17]:
# primero vamos a crear una lista, indicando cuales son las 
# variables a sustituir con cada método

features_numeric = ['BsmtUnfSF', 'LotFrontage', 'MasVnrArea', ]
features_categoric = ['BsmtQual', 'FireplaceQu', 'MSZoning',
                      'Street', 'Alley']

# luego vamos a instanciar imputers dentro de un pipeline
# creamos un imputer por cada variable
# indicando uno para la media y el otro para la categoría más frecuente
# cambiando el parámetro en 'strategy'

numeric_imputer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
])

categoric_imputer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
])

# luego ponemos las variables en lista y los transformadores juntos
# usando el ColumnTransformer

preprocessor = ColumnTransformer(transformers=[
    ('numeric_imputer', numeric_imputer, features_numeric),
    ('categoric_imputer', categoric_imputer, features_categoric)
])

In [18]:
# ajustemos el preprocesador
preprocessor.fit(X_train)

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('numeric_imputer',
                                 Pipeline(memory=None,
                                          steps=[('imputer',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='mean',
                                                                verbose=0))],
                                          verbose=False),
                                 ['BsmtUnfSF', 'LotFrontage', 'MasVnrArea']),
                                ('categoric_imputer',
                                 Pipeline(m

In [19]:
# exploremos los transformers:

preprocessor.transformers

[('numeric_imputer', Pipeline(memory=None,
           steps=[('imputer',
                   SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                                 missing_values=nan, strategy='mean',
                                 verbose=0))],
           verbose=False), ['BsmtUnfSF', 'LotFrontage', 'MasVnrArea']),
 ('categoric_imputer', Pipeline(memory=None,
           steps=[('imputer',
                   SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                                 missing_values=nan, strategy='most_frequent',
                                 verbose=0))],
           verbose=False), ['BsmtQual',
   'FireplaceQu',
   'MSZoning',
   'Street',
   'Alley'])]

In [20]:
# exploremos los parámetros aprendidos:

# para el imputer de la media ( variables numéricas)

preprocessor.named_transformers_['numeric_imputer'].named_steps['imputer'].statistics_

array([565.99217221,  69.66866747, 103.55358899])

In [21]:
# corroboremos el valor para cada variable en el segmento de entrenamiento
X_train[features_numeric].mean()

BsmtUnfSF      565.992172
LotFrontage     69.668667
MasVnrArea     103.553589
dtype: float64

In [22]:
# imputer para la categoría más frecuente
preprocessor.named_transformers_['categoric_imputer'].named_steps['imputer'].statistics_

array(['TA', 'Gd', 'RL', 'Pave', 'Pave'], dtype=object)

In [23]:
# corroboremos estos valores en el segmento de entrenamiento
X_train[features_categoric].mode()

,BsmtQual,FireplaceQu,MSZoning,Street,Alley
0,TA,Gd,RL,Pave,Pave


In [24]:
# y ahora sustituyamos en los segmentos de prueba y entrenamiento
X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

In [25]:
X_train.shape

(1022, 8)

In [26]:
# ahora veamos el resultado de la imputación en el dataframe de 3 columnas
pd.DataFrame(X_train,
             columns=features_numeric + features_categoric).head()

,BsmtUnfSF,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,MSZoning,Street,Alley
0,318,69.6687,573,Gd,Gd,RL,Pave,Pave
1,288,69.6687,0,Gd,Gd,RL,Pave,Pave
2,162,50,0,TA,Gd,RL,Pave,Pave
3,356,60,0,TA,Gd,RL,Pave,Pave
4,0,60,0,TA,Gd,RL,Pave,Pave


En este caso, pasamos todas las variables disponibles en los datos a los imputers de valores ausentes, por lo tanto el dataframe final contiene todas las variables